# Self-Attention機構のデモ

このノートブックでは、Transformerの核となる**Self-Attention（自己注意機構）**をゼロから実装し、その動作を理解します。

## 学習目標

1. Self-Attentionの計算フロー（Q, K, V → Attention Scores → Softmax → 出力）を理解する
2. Attention Weightsを可視化して、どの要素が他の要素に注目しているかを確認する
3. 実装したSelf-Attentionを使って簡単なタスクを実行する

## Attention機構とは？

Attention機構は、入力シーケンスの各要素が他の要素とどれだけ関連しているかを計算し、重要な情報に「注意」を向ける仕組みです。

**数式**: `Attention(Q, K, V) = softmax(QK^T / √d_k) V`

- **Q (Query)**: 「何を探しているか」
- **K (Key)**: 「何を持っているか」
- **V (Value)**: 「実際の情報」

## 1. 必要なライブラリのインポート

In [ ]:
import sys
sys.path.append('../src')  # srcディレクトリをパスに追加

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from attention import SelfAttention, ScaledDotProductAttention

# 日本語フォントの設定（グラフ表示用）
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Hiragino Sans']

# シード値の設定（再現性のため）
torch.manual_seed(42)
np.random.seed(42)

# デバイスの設定（macOS GPU対応）
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"使用デバイス: CUDA GPU")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"使用デバイス: Apple Silicon GPU (MPS)")
else:
    device = torch.device("cpu")
    print(f"使用デバイス: CPU")

print(f"PyTorch version: {torch.__version__}")

使用デバイス: cpu


## 2. 入力データの準備

簡単な例として、4つの単語からなる文を考えます。各単語は8次元のベクトルで表現されます。

実際の自然言語処理では、単語埋め込み（Word Embedding）を使いますが、ここではランダムなベクトルを使用します。

In [ ]:
# ハイパーパラメータ
seq_len = 4      # シーケンス長（単語数）
d_model = 8      # 各単語の埋め込み次元数
batch_size = 1   # バッチサイズ

# 入力データの生成（ランダムな単語埋め込み）
# 形状: [batch_size, seq_len, d_model]
X = torch.randn(batch_size, seq_len, d_model)

print(f"入力データの形状: {X.shape}")
print(f"  - バッチサイズ: {batch_size}")
print(f"  - シーケンス長: {seq_len} (単語数)")
print(f"  - 埋め込み次元: {d_model}")
print(f"\n入力データ (最初のサンプル):")
print(X[0].numpy())

# 可視化のため、単語に名前をつける
words = ["私は", "猫が", "好き", "です"]
print(f"\n単語: {words}")

## 3. Query、Key、Valueの計算

Self-Attentionでは、入力から3つの異なる表現を作ります：

- **Query (Q)**: 「この単語は何を探しているか？」
- **Key (K)**: 「この単語は何についての情報を持っているか？」
- **Value (V)**: 「この単語の実際の情報」

これらは入力を線形変換（重み行列との積）で生成します。

In [ ]:
# Q, K, Vを生成するための線形変換層
W_q = nn.Linear(d_model, d_model, bias=False)
W_k = nn.Linear(d_model, d_model, bias=False)
W_v = nn.Linear(d_model, d_model, bias=False)

# Q, K, Vの計算
Q = W_q(X)  # Query: [batch, seq_len, d_model]
K = W_k(X)  # Key: [batch, seq_len, d_model]
V = W_v(X)  # Value: [batch, seq_len, d_model]

print(f"Query (Q) の形状: {Q.shape}")
print(f"Key (K) の形状: {K.shape}")
print(f"Value (V) の形状: {V.shape}")
print(f"\n重要なポイント:")
print(f"  - Q, K, V はすべて同じ入力 X から生成される → 「Self」Attention")
print(f"  - それぞれ異なる重み行列 W_q, W_k, W_v で変換される")
print(f"  - Q と K の内積で「関連度」を計算し、V を重み付けする")

## 4. Attention Scoresの計算

各単語（Query）が他の全ての単語（Key）とどれだけ関連しているかを内積で計算します。

**計算式**: `scores = Q × K^T / √d_k`

- 内積が大きい = 関連度が高い
- √d_k でスケーリング = 値が大きくなりすぎるのを防ぐ（勾配安定化）

In [ ]:
# Step 1: Q と K^T の内積を計算
# Q: [batch, seq_len, d_model]
# K.transpose(-2, -1): [batch, d_model, seq_len]
# scores: [batch, seq_len, seq_len]
scores = torch.matmul(Q, K.transpose(-2, -1))

print(f"内積スコアの形状: {scores.shape}")
print(f"  - 各行: 1つの単語（Query）")
print(f"  - 各列: 全ての単語（Key）との関連度")

# Step 2: √d_k でスケーリング
d_k = Q.size(-1)  # d_modelと同じ
scores_scaled = scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

print(f"\nスケーリング前のスコア:")
print(scores[0].detach().numpy())
print(f"\nスケーリング後のスコア (÷ √{d_k} = ÷ {np.sqrt(d_k):.2f}):")
print(scores_scaled[0].detach().numpy())

## 5. Softmaxによる正規化

スコアにSoftmax関数を適用して、確率分布（合計が1）に変換します。

これにより、各単語が他の全単語に対してどれだけ「注意」を払うべきかの重みが得られます。

In [ ]:
# Softmaxを適用（最後の次元=各行について正規化）
attention_weights = F.softmax(scores_scaled, dim=-1)

print(f"Attention Weightsの形状: {attention_weights.shape}")
print(f"\nAttention Weights (最初のサンプル):")
print(attention_weights[0].detach().numpy())

# 各行の合計が1になることを確認
row_sums = attention_weights[0].sum(dim=-1)
print(f"\n各行の合計（すべて1.0になるはず）:")
print(row_sums.detach().numpy())

# 意味の解釈
print(f"\n【解釈例】1行目を見ると:")
print(f"  単語 '{words[0]}' は、")
for i, w in enumerate(words):
    weight = attention_weights[0, 0, i].item()
    print(f"    - '{w}' に {weight:.3f} (={weight*100:.1f}%) 注意を払っている")

## 6. Attention Weightsの可視化

ヒートマップでAttention Weightsを可視化します。

- 縦軸: Query（注意を払う側の単語）
- 横軸: Key（注意を受ける側の単語）
- 色の濃さ: 注意の強さ（明るいほど強い関連）

In [ ]:
# Attention Weightsのヒートマップ
plt.figure(figsize=(8, 6))
sns.heatmap(
    attention_weights[0].detach().numpy(),
    annot=True,           # 数値を表示
    fmt='.3f',            # 小数点以下3桁
    cmap='YlOrRd',        # カラーマップ（黄色→オレンジ→赤）
    xticklabels=words,    # x軸のラベル
    yticklabels=words,    # y軸のラベル
    cbar_kws={'label': 'Attention Weight'}
)
plt.xlabel('Key (注意を受ける側)', fontsize=12)
plt.ylabel('Query (注意を払う側)', fontsize=12)
plt.title('Self-Attention Weights のヒートマップ', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("【読み方】")
print("- 各行を見ると、その単語が他の単語にどれだけ注意を払っているかがわかる")
print("- 対角成分は、各単語が自分自身にどれだけ注意を払っているか")

## 7. 出力の計算

Attention WeightsとValueの重み付き和を取り、Self-Attentionの最終出力を得ます。

**計算式**: `Output = Attention_Weights × V`

各単語の出力は、全単語のValue表現を、Attention Weightsで重み付けして足し合わせたものになります。

In [ ]:
# Attention WeightsとValueの積
# attention_weights: [batch, seq_len, seq_len]
# V: [batch, seq_len, d_model]
# output: [batch, seq_len, d_model]
output = torch.matmul(attention_weights, V)

print(f"出力の形状: {output.shape}")
print(f"  - 入力と同じ形状 [batch_size, seq_len, d_model]")
print(f"\n出力データ (最初のサンプル):")
print(output[0].detach().numpy())

print(f"\n【重要な理解】")
print(f"各単語の出力ベクトルは、全単語のValue表現の重み付き和")
print(f"  例: 1番目の単語 '{words[0]}' の出力 = ")
print(f"      {attention_weights[0,0,0]:.3f} × V[0] + {attention_weights[0,0,1]:.3f} × V[1] + ...")
print(f"\nこれにより、各単語が文脈情報（他の単語の情報）を取り込んだ表現になる！")

## 8. 実装したSelf-Attentionクラスを使う

ここまでステップバイステップで計算してきた内容を、`src/attention.py`で実装済みの`SelfAttention`クラスを使って再現します。

In [ ]:
# Self-Attentionモデルのインスタンス化
model = SelfAttention(d_model=d_model, dropout=0.0)
model.eval()  # 評価モード（ドロップアウトを無効化）

# 新しい入力データで実行
X_test = torch.randn(1, seq_len, d_model)
output_test, attention_weights_test = model(X_test)

print(f"クラスを使った場合の出力形状: {output_test.shape}")
print(f"Attention Weights形状: {attention_weights_test.shape}")

# 可視化
plt.figure(figsize=(8, 6))
sns.heatmap(
    attention_weights_test[0].detach().numpy(),
    annot=True,
    fmt='.3f',
    cmap='Blues',
    xticklabels=words,
    yticklabels=words,
    cbar_kws={'label': 'Attention Weight'}
)
plt.xlabel('Key', fontsize=12)
plt.ylabel('Query', fontsize=12)
plt.title('SelfAttentionクラスによる Attention Weights', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

## 9. 簡単な学習タスク：数列のコピー

Self-Attentionが実際に学習できることを確認するため、シンプルなタスクを試します。

**タスク**: 入力数列をそのままコピーする

例: `[1, 2, 3, 4]` → `[1, 2, 3, 4]`

これは簡単なタスクですが、Self-Attentionがシーケンス情報を学習できることを示します。

In [ ]:
# シンプルなモデル: Self-Attention + 線形層
class SimpleSelfAttentionModel(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.attention = SelfAttention(d_model, dropout=0.1)
        self.output_layer = nn.Linear(d_model, d_model)
        
    def forward(self, x):
        # Self-Attentionを適用
        attn_out, attn_weights = self.attention(x)
        # 線形層で出力
        output = self.output_layer(attn_out)
        return output, attn_weights

# モデルのインスタンス化
d_model_task = 16
model_task = SimpleSelfAttentionModel(d_model_task).to(device)
optimizer = torch.optim.Adam(model_task.parameters(), lr=0.01)
criterion = nn.MSELoss()

# 訓練データの生成（ランダムなベクトル列をコピーするタスク）
def generate_copy_data(batch_size, seq_len, d_model):
    """入力と同じ出力を返すデータを生成"""
    x = torch.randn(batch_size, seq_len, d_model)
    y = x.clone()  # コピータスク
    return x, y

# 学習ループ
num_epochs = 100
seq_len_task = 6
batch_size_task = 32
losses = []

print("学習開始...")
for epoch in range(num_epochs):
    # データ生成
    x_train, y_train = generate_copy_data(batch_size_task, seq_len_task, d_model_task)
    x_train, y_train = x_train.to(device), y_train.to(device)
    
    # 順伝播
    optimizer.zero_grad()
    output, _ = model_task(x_train)
    
    # 損失計算
    loss = criterion(output, y_train)
    
    # 逆伝播と最適化
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")

print("学習完了！")

# 損失の可視化
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('学習曲線: 数列コピータスク')
plt.grid(True, alpha=0.3)
plt.show()

## 10. テスト：学習したモデルの評価

学習したモデルがコピータスクをちゃんと実行できるか確認します。

In [ ]:
# テストデータで評価
model_task.eval()
with torch.no_grad():
    x_test, y_test = generate_copy_data(1, seq_len_task, d_model_task)
    x_test, y_test = x_test.to(device), y_test.to(device)
    
    output_pred, attn_weights_final = model_task(x_test)
    
    test_loss = criterion(output_pred, y_test)
    
    print(f"テスト損失: {test_loss.item():.6f}")
    print(f"\n入力ベクトルの一部:")
    print(x_test[0, :3, :5].cpu().numpy())  # 最初の3単語、5次元まで表示
    print(f"\n期待される出力（= 入力のコピー）:")
    print(y_test[0, :3, :5].cpu().numpy())
    print(f"\nモデルの予測出力:")
    print(output_pred[0, :3, :5].cpu().numpy())
    print(f"\n→ かなり近い値を予測できています！")

# Attention Weightsの可視化
plt.figure(figsize=(8, 6))
sns.heatmap(
    attn_weights_final[0].cpu().numpy(),
    annot=True,
    fmt='.2f',
    cmap='Greens',
    cbar_kws={'label': 'Attention Weight'}
)
plt.xlabel('Key (位置)', fontsize=12)
plt.ylabel('Query (位置)', fontsize=12)
plt.title('学習後のAttention Weights（コピータスク）', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("\n【Attention Weightsの解釈】")
print("対角成分が強い = 各位置が自分自身に強く注意を払っている")
print("→ コピータスクでは、各位置の情報をそのまま出力すれば良いため、この傾向は理にかなっています")

## まとめ

このノートブックでは、Self-Attention機構について以下を学びました：

### 理論
1. **Query, Key, Value**: 入力から3つの異なる表現を生成
2. **Attention Scores**: Q と K の内積で関連度を計算
3. **Softmax**: スコアを確率分布に変換
4. **重み付き和**: Attention WeightsとValueで最終出力を計算

### 実装
- PyTorchで`SelfAttention`クラスを実装
- Attention Weightsをヒートマップで可視化
- 簡単なコピータスクで学習を確認

### 次のステップ
- **Multi-Head Attention**: 複数のAttentionを並列に実行
- **Position Encoding**: シーケンスの順序情報を追加
- **Transformer Encoder**: Self-Attention + Feed Forward Network

Self-Attentionは、Transformerの核となる強力な機構です。この理解を土台に、次はMulti-Head Attentionを実装していきましょう！